# Gas Particle Simulation

This notebook demonstrates a simple 2D simulation of gas particles in a box. The simulation:
- Shows particles bouncing off walls (elastic collisions)
- Implements particle-particle collisions
- Visualizes the motion in real-time
- Tracks the system's kinetic energy

## Import Required Libraries

We'll need NumPy for calculations and Matplotlib for visualization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.patches as patches

%matplotlib inline

## Simulation Parameters

Define the parameters for our simulation. Feel free to adjust these values to see different behaviors.

In [ ]:
# Simulation parameters
num_particles = 50  # Number of particles
box_size = 10.0     # Box dimensions (square)
particle_radius = 0.2
max_speed = 0.1     # Maximum initial speed

## Initialize Particle Properties

Create arrays for particle positions, velocities, and masses with random initial values.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Particle properties: position, velocity, mass
positions = np.random.uniform(particle_radius, box_size - particle_radius, size=(num_particles, 2))
velocities = np.random.uniform(-max_speed, max_speed, size=(num_particles, 2))
masses = np.ones(num_particles)  # All particles have the same mass

print(f"Initialized {num_particles} particles in a {box_size}x{box_size} box")
print(f"Average initial speed: {np.mean(np.sqrt(velocities[:,0]**2 + velocities[:,1]**2)):.4f}")

## Function for Handling Collisions

This function handles both wall collisions and particle-particle collisions.

In [ ]:
def handle_collisions(positions, velocities, masses, box_size, particle_radius):
    """
    Handle collisions between particles and with walls
    """
    # Check for collisions with walls
    # Left and right walls
    wall_collision_x = (positions[:, 0] <= particle_radius) | (positions[:, 0] >= box_size - particle_radius)
    velocities[wall_collision_x, 0] *= -1  # Reverse x velocity
    
    # Top and bottom walls
    wall_collision_y = (positions[:, 1] <= particle_radius) | (positions[:, 1] >= box_size - particle_radius)
    velocities[wall_collision_y, 1] *= -1  # Reverse y velocity
    
    # Keep particles inside the box
    positions[:, 0] = np.clip(positions[:, 0], particle_radius, box_size - particle_radius)
    positions[:, 1] = np.clip(positions[:, 1], particle_radius, box_size - particle_radius)
    
    # Check for collisions between particles
    for i in range(len(positions)):
        for j in range(i+1, len(positions)):
            # Calculate distance between particles
            dx = positions[j, 0] - positions[i, 0]
            dy = positions[j, 1] - positions[i, 1]
            distance = np.sqrt(dx**2 + dy**2)
            
            # Check if particles overlap (collision)
            if distance <= 2 * particle_radius:
                # Collision handling with elastic collision physics
                # Normal direction
                nx = dx / distance
                ny = dy / distance
                
                # Relative velocity
                dvx = velocities[j, 0] - velocities[i, 0]
                dvy = velocities[j, 1] - velocities[i, 1]
                
                # Project velocity onto normal direction
                vn = dvx * nx + dvy * ny
                
                # Ignore if particles are moving away from each other
                if vn < 0:
                    # Impulse formula for elastic collision
                    impulse = 2 * vn / (1/masses[i] + 1/masses[j])
                    
                    # Apply impulse
                    velocities[i, 0] += impulse * nx / masses[i]
                    velocities[i, 1] += impulse * ny / masses[i]
                    velocities[j, 0] -= impulse * nx / masses[j]
                    velocities[j, 1] -= impulse * ny / masses[j]
                    
                    # Move particles slightly to avoid overlap
                    overlap = 2 * particle_radius - distance
                    positions[i, 0] -= overlap * nx / 2
                    positions[i, 1] -= overlap * ny / 2
                    positions[j, 0] += overlap * nx / 2
                    positions[j, 1] += overlap * ny / 2
    
    return positions, velocities

## Simulation Function

This function advances the simulation by one time step and returns the updated positions.

In [ ]:
def update_simulation(positions, velocities, masses, box_size, particle_radius):
    """
    Update particle positions and handle collisions
    """
    # Update positions
    positions += velocities
    
    # Handle collisions
    positions, velocities = handle_collisions(positions, velocities, masses, box_size, particle_radius)
    
    # Calculate kinetic energy
    kinetic_energy = 0.5 * np.sum(masses[:, np.newaxis] * velocities**2)
    
    return positions.copy(), velocities.copy(), kinetic_energy

## Visualization Setup

Now let's set up the visualization for our particle simulation. We'll use matplotlib to draw the box and particles.

In [ ]:
def setup_visualization(positions, box_size, particle_radius):
    """Set up the figure and axes for visualization"""
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(0, box_size)
    ax.set_ylim(0, box_size)
    ax.set_aspect('equal')
    ax.set_title('Gas Particle Simulation')
    
    # Create box borders
    box = patches.Rectangle((0, 0), box_size, box_size, linewidth=2, 
                          edgecolor='black', facecolor='none')
    ax.add_patch(box)
    
    # Create particles (circles)
    circles = [plt.Circle((positions[i, 0], positions[i, 1]), 
                         particle_radius, color='blue', alpha=0.7) 
              for i in range(len(positions))]
    
    for circle in circles:
        ax.add_patch(circle)
    
    # Text for displaying kinetic energy
    energy_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    
    return fig, ax, circles, energy_text

## Animation Function

This function updates the animation frame by frame. For Jupyter notebooks, we need to use a special animation approach.

In [ ]:
def animate_gas_particles():
    # Create a copy of the initial state
    pos = positions.copy()
    vel = velocities.copy()
    
    # Set up visualization
    fig, ax, circles, energy_text = setup_visualization(pos, box_size, particle_radius)
    
    def update(frame):
        nonlocal pos, vel
        
        # Update simulation
        pos, vel, kinetic_energy = update_simulation(pos, vel, masses, box_size, particle_radius)
        
        # Update circle positions
        for i, circle in enumerate(circles):
            circle.center = pos[i, 0], pos[i, 1]
        
        # Update energy text
        energy_text.set_text(f'Kinetic Energy: {kinetic_energy:.2f}')
        
        return circles + [energy_text]
    
    # Create animation
    anim = FuncAnimation(fig, update, frames=200, interval=20, blit=True)
    plt.close()  # Prevent duplicate display in Jupyter
    
    # Display the animation
    from IPython.display import HTML
    return HTML(anim.to_jshtml())

# Run the animation
animate_gas_particles()

## Analyzing the Physics

Let's run the simulation for many steps and track the kinetic energy over time to analyze the system behavior.

In [ ]:
def run_simulation_analysis(steps=1000):
    # Reset to initial conditions
    pos = positions.copy()
    vel = velocities.copy()
    
    # Arrays to store data
    kinetic_energies = np.zeros(steps)
    velocities_distribution = []
    
    # Run simulation
    for step in range(steps):
        pos, vel, kinetic_energies[step] = update_simulation(pos, vel, masses, box_size, particle_radius)
        
        # Every 100 steps, record velocity distribution
        if step % 100 == 0:
            speeds = np.sqrt(vel[:,0]**2 + vel[:,1]**2)
            velocities_distribution.append(speeds)
    
    # Plot kinetic energy over time
    plt.figure(figsize=(10, 4))
    plt.plot(kinetic_energies)
    plt.xlabel('Time Step')
    plt.ylabel('Kinetic Energy')
    plt.title('Kinetic Energy Conservation')
    plt.grid(True)
    plt.show()
    
    # Plot final velocity distribution
    plt.figure(figsize=(8, 5))
    plt.hist(velocities_distribution[-1], bins=15, alpha=0.7)
    plt.xlabel('Particle Speed')
    plt.ylabel('Count')
    plt.title('Maxwell-Boltzmann Speed Distribution')
    plt.grid(True)
    plt.show()
    
    return {
        'avg_kinetic_energy': np.mean(kinetic_energies),
        'std_kinetic_energy': np.std(kinetic_energies),
        'final_positions': pos,
        'final_velocities': vel
    }

# Run the analysis
results = run_simulation_analysis(steps=500)
print(f"Average Kinetic Energy: {results['avg_kinetic_energy']:.4f} ± {results['std_kinetic_energy']:.4f}")

## Conclusion

This simulation demonstrates several key principles of statistical mechanics:

1. **Conservation of Energy**: The total kinetic energy remains approximately constant
2. **Maxwell-Boltzmann Distribution**: The distribution of particle speeds approaches this theoretical distribution
3. **Pressure**: The particles exert pressure on the walls through collisions

You can experiment with different parameters like:
- Changing the number of particles
- Varying particle sizes
- Adjusting the initial velocity distribution
- Adding external forces (like gravity)

These modifications would allow you to simulate different physical systems and phenomena.